In [ ]:
%load_ext autoreload

%autoreload 2

import open3d as o3d
import os
import numpy as np
import torch
from torch_geometric import transforms as T

from tbp.monty.frameworks.environments.modelnet import ModelNet40

#### Start by replicating the basic results in the open3d tutorial

They have a tutorial I was trying to follow on poisson surface reconstruction, but to get stuff working I had to uninstall open3d and install using pip instead of conda to get version 0.15.0. Apparently a lot has changed with open3d in the last few versions, and for whatever reason, the conda command I used previously did not give me the updated version. It doesn't help that there are many older versions of their documentation floating around

http://www.open3d.org/docs/latest/getting_started.html#development-version-pip
http://www.open3d.org/docs/release/introduction.html
http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html

To check you have the right version, `import open3d as o3d; o3d.__version__; print(o3d.data)`. You should have version 0.15.0 and there should be a submodule `data`, which does not exist in 0.11 which is what I had originally.

In [ ]:
dataset = o3d.data.EaglePointCloud()
pcd = o3d.io.read_point_cloud(dataset.path)
o3d.visualization.draw(pcd)

In [ ]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

#### OK, now try taking a modelnet40 object, getting a numpy list of 3d points, passing to open3d pointcloud class, using PC.estimate_normals, and then poisson reconstruction

In [ ]:
class RemoveFaces(T.BaseTransform):
    def __call__(self, data):
        return data.pos


class ToNumpy(T.BaseTransform):
    def __call__(self, data):
        return data.numpy()


In [ ]:
transform = T.Compose([
        T.NormalizeScale(),
        T.SamplePoints(num=1024),
        RemoveFaces()
    ])

In [ ]:
dataset = ModelNet40(
    root=os.path.expanduser("~/tbp/datasets/ModelNet40/raw"),
    transform=transform,
    train=True,
    num_samples_train=2)

In [ ]:
xy = dataset[6]

In [ ]:
print(xy)

In [ ]:
x = xy[0]

In [ ]:
x.size()

In [ ]:
dataset.id_to_object

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)

In [ ]:
pcd

In [ ]:
pcd.normals = o3d.utility.Vector3dVector(np.zeros(
    (1, 3)))  # invalidate existing normals

pcd.estimate_normals()

In [ ]:
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

In [ ]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh])
                                #   zoom=0.664,
                                #   front=[-0.4761, -0.4698, -0.7434],
                                #   lookat=[1.8900, 3.2596, 0.9284],
                                #   up=[0.2304, -0.8825, 0.4101])

#### OK, hard to tell what I'm looking at

Maybe it depends on the number of points we are sampling from the mesh. Try, various powers of 2

In [ ]:
def modelnet40_to_poisson_surface(obj):

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(obj)
    pcd.normals = o3d.utility.Vector3dVector(np.zeros(
    (1, 3)))  # invalidate existing normals
    pcd.estimate_normals()
    print('run Poisson surface reconstruction')
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
    
    return mesh
    # o3d.visualization.draw_geometries([mesh])

In [ ]:
dataset = ModelNet40(
    root=os.path.expanduser("~/tbp/datasets/ModelNet40/raw"),
    transform=None,
    train=True,
    num_samples_train=2
)

X = dataset.data[10]

for num in [2**7, 2**10, 2**13]:

    transform = T.Compose([
            T.NormalizeScale(),
            T.SamplePoints(num=num),
            RemoveFaces()
        ])

    x = transform(X)
    modelnet40_to_poisson_surface(x)

In [ ]:
for num in [2**7, 2**10, 2**13]:

    transform = T.Compose([
            T.NormalizeScale(),
            T.SamplePoints(num=num),
            RemoveFaces()
        ])

    dataset = ModelNet40(
        root=os.path.expanduser("~/tbp/datasets/ModelNet40/raw"),
        transform=transform,
        train=True,
        num_samples_train=2
    )
    x = dataset[10]
    modelnet40_to_poisson_surface(x)

In [ ]:
# dataset = ModelNet40(
#     root=os.path.expanduser("~/tbp/datasets/ModelNet40/raw"),
#     transform=None,
#     train=True,
#     num_samples_train=2
# )

# x = dataset.data[10]


# for num in [2**7, 2**10, 2**13]:

#     transform = T.Compose([
#             T.NormalizeScale(),
#             T.SamplePoints(num=num),
#             RemoveFaces()
#         ])

#     x_ = transform(x)
#     modelnet40_to_poisson_surface(x_)
    
    